# ДЗ №4
### *виконала студентка групи ФБ-33 Журавльова Марія* ###

Нагадаю, що мій набір даних - TMDB 10000 Movies Dataset (https://www.kaggle.com/datasets/i0xc0d3x00000/tmdb-10000-movies-dataset)

**Опис колонок:**

- title - назва фільму;
- overview - опис сюжету фільму;
- release_date - оригінальна дата випуску;
- vote_average - середній рейтинг фільму;
- vote_count - кількість отриманих голосів;
- original_language - мова зйомки;
- popularity - індекс популярності.

**Прокласифікувати методом К найближчих судідів:**


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Завантаження даних
df = pd.read_csv(r"C:\Users\User\OneDrive\Desktop\university\МТАД\movie_tmdb_clean.csv")

# Визначення "успішного" фільму
df['label'] = (df['vote_average'] >= 8).astype(int)

# Визначення ознак
X = df[['vote_count', 'popularity']]
Y = df['label']

# Стандартизація
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

- Поділити вибірку на тестову та начальну;

In [2]:
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.3, random_state=100)

- Розрахувати скільки потрібно обрати найближчих записів, щоб точність була не нижче 85%;
- Порахувати точність для тестової вибірки;

In [3]:
optimal_k = 0
best_acc = 0
results_data = []

for k in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, Y_train)
    y_pred = knn.predict(X_test)
    acc = accuracy_score(Y_test, y_pred)
    results_data.append({'K': k, 'Accuracy': acc})
    if acc >= 0.85 and acc > best_acc:
        best_acc = acc
        optimal_k = k

df_results = pd.DataFrame(results_data)
df_results['Accuracy'] = df_results['Accuracy'].map('{:.3f}'.format)
print(df_results)
print(f"\nОптимальне K={optimal_k}")
print(f"Точність текстової вибірки при K={optimal_k}: {best_acc:.3f}")

     K Accuracy
0    1    0.946
1    2    0.972
2    3    0.968
3    4    0.972
4    5    0.972
5    6    0.972
6    7    0.972
7    8    0.972
8    9    0.972
9   10    0.972
10  11    0.972
11  12    0.972
12  13    0.972
13  14    0.972
14  15    0.972
15  16    0.972
16  17    0.972
17  18    0.972
18  19    0.972
19  20    0.972

Оптимальне K=2
Точність текстової вибірки при K=2: 0.972


- Зробити крос валідацію на 3 блоках;

In [4]:
knn = KNeighborsClassifier(n_neighbors=optimal_k)
cv_scores = cross_val_score(knn, X_scaled, Y, cv=3, scoring='accuracy')
print("Результати:")
for i, score in enumerate(cv_scores, 1):
    print (f" - Блок {i:<2} : {score:.3f}")
print(f"Середня точність крос-валідації: {cv_scores.mean():.3f}")
print(f"Стандартне відхилення: {cv_scores.std():.3f}")

Результати:
 - Блок 1  : 0.970
 - Блок 2  : 0.971
 - Блок 3  : 0.970
Середня точність крос-валідації: 0.970
Стандартне відхилення: 0.000


**Висновки.** Метод KNN успішно класифікував фільми на "успішні" та "неуспішні". Оптимальним значенням було обрано K=2 як найменше значення K, яке дає максимальну точність на текстовій виборці. Результати крос-валідації показали високу середню точність, що вказує на те, що модель KNN з K=2 має високу узагальнюючу здатність. Стандартне відхилення крос-валідації становить 0, що свідчить про ідеальну стабільність моделі.